In [43]:
cd(@__DIR__)
using Pkg
Pkg.activate(".")
Pkg.instantiate()

 Activating environment at `/Volumes/MyFiles/GitRepos/StatisticalMice/ProjectsPublic/Benchmarks/Project.toml`


In [44]:
Pkg.status()

Status `/Volumes/MyFiles/GitRepos/StatisticalMice/ProjectsPublic/Benchmarks/Project.toml`
  [6e4b80f9] BenchmarkTools v0.5.0
  [a93c6f00] DataFrames v0.22.4
  [31c24e10] Distributions v0.24.12
  [bdcacae8] LoopVectorization v0.10.0
  [91a5bcdd] Plots v1.10.2
  [b0e4dd01] RollingFunctions v0.6.2
  [c6e7a17c] SantasLittleHelpers v0.2.1
  [4c63dfa8] StaticKernels v0.5.1
  [ac1d9e8a] ThreadsX v0.1.7
  [9ff05d80] TickTock v1.1.0
  [bc48ee85] Tullio v0.2.11
  [9a3f8284] Random
  [10745b16] Statistics


In [45]:
# Set some notebook defaults
ENV["COLUMNS"] = 1000; ENV["LINES"] = 20;

# Display information about the environment
VERSION, Base.Threads.nthreads(), Base.Sys.CPU_THREADS

(v"1.5.3", 1, 8)

In [46]:
using TickTock
tick()

┌ Info:  started timer at: 2021-01-27T11:36:31.63
└ @ TickTock /Users/arihuttunen/.julia/packages/TickTock/RsTHR/src/TickTock.jl:32


In [47]:
N = 10^8

100000000

In [48]:
using Random, Distributions, BenchmarkTools, DataFrames

In [49]:
d = Normal()
df = DataFrame(:data => rand(d, N));

In [50]:
out = Vector{Union{Missing, Float64}}(undef, N)

function rollingmean2(x, m, out)
    N = length(x)
    out[1:(m-1)] .= missing
    sm = mean(@view x[1:m])
    out[m] = sm
    @inbounds @simd for i in (m+1):length(x)
        sm += (x[i] - x[i-m]) / m
        out[i] = sm
    end
    return out 
end

rollingmean2 (generic function with 1 method)

In [51]:
df[!, "sma"] = rollingmean2(df.data, 10, out)
df

,data,sma
,Float64,Float64?
1,1.72455,missing
2,-1.56299,missing
3,0.38958,missing
4,0.0312338,missing
5,0.466341,missing
6,0.328187,missing
7,0.451401,missing
8,-0.631133,missing
9,0.0134446,missing


In [52]:
tock()

┌ Info:           3.59660456s: 3 seconds, 596 milliseconds
└ @ TickTock /Users/arihuttunen/.julia/packages/TickTock/RsTHR/src/TickTock.jl:39


In [53]:
@benchmark rand(d, N)

BenchmarkTools.Trial: 
  memory estimate:  762.94 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.205 s (0.07% GC)
  median time:      1.361 s (0.08% GC)
  mean time:        1.331 s (1.88% GC)
  maximum time:     1.396 s (6.96% GC)
  --------------
  samples:          4
  evals/sample:     1

In [54]:
@benchmark rollingmean2(df.data, 10, out)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     115.358 ms (0.00% GC)
  median time:      134.893 ms (0.00% GC)
  mean time:        133.694 ms (0.00% GC)
  maximum time:     148.640 ms (0.00% GC)
  --------------
  samples:          38
  evals/sample:     1